Moved this to a separate file (i.e. once OCR is done, I don't need to keep re-running it for tests)

In [1]:
%pip install -q \
    pandas \
    prisma

Note: you may need to restart the kernel to use updated packages.


In [22]:
#--schema=../receipts-app/prisma/schema.prisma
# !npm install -g prisma
# !prisma generate

# Setting up prisma python client was pretty hard. For now I copy the schema and .env file from the Next.js app folder
# then source .venv/bin/activate ... and run the prisma commands from there, including installing prisma, generate, and migrate
!cp ../receipts-app/prisma/dev.db ./prisma/dev.db
!cp ../receipts-app/prisma/schema.prisma ./prisma/schema.prisma
!perl -i -pe's/prisma-client-js/prisma-client-py/g' ./prisma/schema.prisma
!cp ../receipts-app/.env ./.env

In [1]:
import pandas as pd
import asyncio

In [2]:
# grab the CSVs with results from the OCR to save time
receipts_df = pd.read_csv('receipts.csv')
expenses_df = pd.read_csv('expenses.csv')

In [5]:
# # at first I tried with sqlite3 but I might as well use Prisma on both ends

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')
# df1 = pd.read_sql_query("SELECT * from UnitOfMeasure", con)

# print(df.head())

# con.close()

# # using sqlite3 directly

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')

# cur = con.cursor()

# # clear data first
# cur.execute("DELETE FROM UnitOfMeasure")

# # seed to sql lite my data UnitOfMeasure
# for unit in ['g', 'mL', 'Count']:
#     cur.execute("INSERT INTO UnitOfMeasure (name) VALUES (?)", (unit,))

# con.commit()
# con.close()


In [6]:
receipts_df

,box,text,confidence
0,"[[np.int32(631), np.int32(269)], [np.int32(806...",2_3,0.040573
1,"[[np.int32(116), np.int32(263)], [np.int32(821...",I~~fs (pke =,0.124128
2,"[[np.int32(405), np.int32(436)], [np.int32(481...","3 """,0.019708
3,"[[np.int32(120), np.int32(447)], [np.int32(317...",Lec|,0.570547
4,"[[np.int32(656), np.int32(493)], [np.int32(900...",'!,0.086648
...,...,...,...
11130,"[[np.int32(464), np.int32(3867)], [np.int32(73...",70104-2707 RTo0o1,0.528192
11131,"[[np.int32(55), np.int32(3892)], [np.int32(501...",THANK You FOR SHOPPING AT 'YIG,0.403911
11132,"[[np.int32(53), np.int32(3931)], [np.int32(259...",STORE MANAGER,0.388632
11133,"[[np.int32(291), np.int32(3929)], [np.int32(53...",SPENCER JONSSON,0.359595


In [7]:
expenses_df

,filename,product,best_match,price
0,img_4601.jpg,Mac & Cheese,Mac & Cheese,512.99
1,img_4601.jpg,Bacon Mac & Cheese,Mac & Cheese,54.25
2,img_4576.jpg,19554 KINN ENGLISH MUFFIN TAPIOCA,Kinn english muffin tapioca,6.35
3,img_4576.jpg,22143 PRANA CASHEWS,Prana cashews gf,13.95
4,img_4576.jpg,18184 FODY TOMATO BASIL SAUCE,Fody Premium Tomato Basil Sauce - 547 mL,9.95
...,...,...,...,...
99,img_4644.jpg,22143 PRANA CASHEWS,Prana cashews gf,13.95
100,img_4644.jpg,28289 ENJOY SEMI-SWEET MEGA CHOC CH,Enjoy Life Mega Chocolate Chunks Semi-Sweet - ...,8.45
101,img_4644.jpg,20236 MAISON ORP OLIVE OIL DELICATE,Maison Orphee Olive Oil Extra Virgin Delicate ...,25.95
102,img_4644.jpg,"29114 LUCKY THAI SPRING ROLLS GF"" 24",Lucky Spring Rolls GF,8.65


In [34]:
# rather do it with prisma, so it's prisma on both sides
# wow, running primsa (npm) with python venv is a ride

from dotenv import load_dotenv
dotenv_path = '../receipts-app/.env'
load_dotenv(dotenv_path)

from prisma import Prisma

async def main():
    # Initialize the Prisma client
    db = Prisma()
    await db.connect()

    # see object attributes (for debug), note that the attributes are in all lowercase here
    # print(dir(db))
    
    # Static data
    unitOfMeasures_df = pd.DataFrame([
        {'name': 'g'},
        {'name': 'mL'},
        {'name': 'Count'}
    ])
    # # Dynamic data, sample for now
    referenceItems = [
        {
            'name': 'Item 1',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 1.99,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item 2',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 2.99,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item A',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 3.99,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item B',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 4.99,
            'referenceUrl': 'https://www.example.com/item1'
        }
    ]
    # receipts = [
    #     {
    #         'filename': 'IMG_4549.jpg',
    #         'receiptTexts': [
    #             {'text': 'Item 1', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.95},
    #             {'text': 'Item 2', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.90},
    #         ]
    #     },
    #     {
    #         'filename': 'IMG_4550.jpg',
    #         'receiptTexts': [
    #             {'text': 'Item A', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.93},
    #             {'text': 'Item B', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.89},
    #         ]
    #     }
    # ]
    # expenses = [
    #     {
    #         'filename': 'IMG_4549.jpg',
    #         'referenceItem': 'Item 1',
    #         'bestMatch': 'Item 1',
    #         'price': 1.99
    #     },
    #     {
    #         'filename': 'IMG_4549.jpg',
    #         'referenceItem': 'Item 2',
    #         'bestMatch': 'Item 2',
    #         'price': 2.99
    #     },
    #     {
    #         'filename': 'IMG_4550.jpg',
    #         'referenceItem': 'Item A',
    #         'bestMatch': 'Item A',
    #         'price': 3.99
    #     },
    #     {
    #         'filename': 'IMG_4550.jpg',
    #         'referenceItem': 'Item B',
    #         'bestMatch': 'Item B',
    #         'price': 4.99
    #     }
    # ]

    # create units of measure
    # first clear db
    await db.referenceitem.delete_many(where={})
    await db.unitofmeasure.delete_many(where={})
    print('Db cleared')

    for unit in unitOfMeasures_df.to_dict(orient='records'):
        # print(unit)
        UoM = await db.unitofmeasure.create(data={'name': unit.get('name')})
        # add UoM.id to unitOfMeasures
        unit['id'] = UoM.id
        # print(f'Unit of Measure created: {UoM.name}')    
    uoms = await db.unitofmeasure.find_many()
    print(uoms)

    # change the referenceItems to use the UoM ids
    for referenceItem in referenceItems:
        # find the UoM id for the unit of measure (next returns None if not found)
        uom_id = next((uom.id for uom in uoms if uom.name == referenceItem['unitOfMeasure']), None)
        if uom_id:
            # Use the connect operation to link to an existing UnitOfMeasure
            referenceItem['unitOfMeasure'] = {
                'connect': {'id': uom_id}
            }

    # create reference items
    for referenceItem in referenceItems:
        refrenceItem_db = await db.referenceitem.create(data=referenceItem)
        referenceItem['id'] = refrenceItem_db.id
    referenceItems_db = await db.referenceitem.find_many()
    print(referenceItems_db)


    # # test create expense
    # expense = await db.expense.create(
    #     data={
    #         'filename': 'IMG_4549.jpg',
    #         'priceEach': 1.99,
    #         'quantity': 1,
    #         ...

    # # create all the reference items
    # for referenceItem in referenceItems:
    #     await db.referenceItem.create(data=referenceItem)

    # Iterate through the receipts and create them along with their texts
    # for receipt_data in receipts:
    #     # Create the Receipt
    #     receipt = await db.receipt.create(
    #         data={
    #             'filename': receipt_data['filename'],
    #             'receiptText': {
    #                 'create': receipt_data['receiptTexts']
    #             }
    #         },
    #         include={
    #             'receiptText': True
    #         }
    #     )
    #     print(f'Receipt created: {receipt.id}, with texts: {[text.text for text in receipt.receiptText]}')

    # Disconnect the Prisma client
    await db.disconnect()

# Run the main function
# asyncio.run(main())
await main()

Db cleared
[UnitOfMeasure(id=34, name='g', ReferenceItem=None), UnitOfMeasure(id=35, name='mL', ReferenceItem=None), UnitOfMeasure(id=36, name='Count', ReferenceItem=None)]
[ReferenceItem(id=13, name='Item 1', quantity=1.0, unitOfMeasureId=36, unitOfMeasure=None, price=1.99, referenceUrl='https://www.example.com/item1', createdAt=datetime.datetime(2024, 7, 7, 2, 43, 4, 102000, tzinfo=TzInfo(UTC)), updatedAt=datetime.datetime(2024, 7, 7, 2, 43, 4, 102000, tzinfo=TzInfo(UTC)), Expense=None), ReferenceItem(id=14, name='Item 2', quantity=1.0, unitOfMeasureId=36, unitOfMeasure=None, price=2.99, referenceUrl='https://www.example.com/item1', createdAt=datetime.datetime(2024, 7, 7, 2, 43, 4, 103000, tzinfo=TzInfo(UTC)), updatedAt=datetime.datetime(2024, 7, 7, 2, 43, 4, 103000, tzinfo=TzInfo(UTC)), Expense=None), ReferenceItem(id=15, name='Item A', quantity=1.0, unitOfMeasureId=36, unitOfMeasure=None, price=3.99, referenceUrl='https://www.example.com/item1', createdAt=datetime.datetime(2024, 7,

In [ ]:
# update Next.js app
!cp ./prisma/dev.db ../receipts-app/prisma/dev.db
!cp ./prisma/schema.prisma ../receipts-app/prisma/schema.prisma
!perl -i -pe's/prisma-client-py/prisma-client-js/g' ../receipts-app/prisma/schema.prisma
!cp ./.env ../receipts-app/.env